<a href="https://colab.research.google.com/github/couplewith/colab/blob/main/%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91%EA%B3%BC_%EB%B6%84%EC%84%9D_AMZN_2408.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 수집과 분석을 자동화하기 위해서는 웹 스크래핑, API 사용, 그리고 자동화된 데이터 분석 도구를 활용할 수 있습니다. 여기서는 각 방법에 대한 상세한 설명과 예제를 제공하겠습니다.

# 주식 데이터 수집과 분석

주식 데이터 수집 : 인터넷 주식 정보 사이트를 통해 주식 정보를 수집하고 분석하여 향후 동향을 분석하는 알고리즘을 만들어 보고자 합니다.


## 1. 데이터 수집

- Yahoo Finance: Yahoo Finance - Amazon
https://finance.yahoo.com/quote/AMZN/history?p=AMZN

- Google Finance: Google Finance - Amazon
https://www.google.com/finance/quote/AMZN:NASDAQ

- Bloomberg: Bloomberg - Amazon
https://www.bloomberg.com/quote/AMZN:US

위의 주식은 다음과 같은 내용을 가지고 있습니다.





In [6]:
 # 파이썬시작
 !pip install requests beautifulsoup4 pandas


## 2.2 데이터수집 -  웹 스크래핑

웹 스크래핑을 통해 웹사이트에서 데이터를 자동으로 수집할 수 있습니다. Python의 BeautifulSoup과 requests 라이브러리를 사용하여 데이터를 수집할 수 있습니다



** 예제: Yahoo Finance에서 데이터 수집 **

In [8]:
#예제: Yahoo Finance에서 데이터 수집 - CSV파일
import pandas as pd

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터프레임 출력
print(df.head())

         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  


In [7]:
#예제: Yahoo Finance에서 데이터 수집 - HTML 파싱
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
url = 'https://finance.yahoo.com/quote/AMZN/history?p=AMZN'

# 요청 보내기
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 데이터 찾기
table = soup.find('table', {'data-test': 'historical-prices'})

# 데이터프레임으로 변환
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

# 데이터 전처리
df = df.dropna()

print(df.head())


AttributeError: 'NoneType' object has no attribute 'find_all'

## 2.2 데이터수집 - API 사용
많은 금융 데이터 제공업체는 API를 통해 데이터를 제공합니다. 예를 들어, Alpha Vantage, Yahoo Finance API, Quandl 등이 있습니다.


** alphavantage API 신청 **
https://www.alphavantage.co/support/#api-key

 Free API 신청으로 하루 25건의 이용이 가능합니다.

 `T5HM0TJESBEDRM94`

** 예제: Alpha Vantage API 사용 **

In [5]:
#예제: Alpha Vantage API 사용
import requests
import pandas as pd

# Alpha Vantage API 키
api_key = 'YOUR_API_KEY'
api_key = 'T5HM0TJESBEDRM94'

# URL 설정
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMZN&apikey={api_key}&outputsize=full&datatype=csv'

# 데이터 요청
response = requests.get(url)
data = response.text

# 데이터프레임으로 변환
df = pd.read_csv(pd.compat.StringIO(data))

# 데이터 전처리
df = df.dropna()

print(df.head())

AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [11]:
!pip install statsmodels pandas requests

In [13]:
import requests
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터프레임 출력
print(df.head())


# 데이터 전처리
df['Close'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 모델 학습
model = ARIMA(df['Close'], order=(5, 1, 0))
model_fit = model.fit()

# 주가 예측
forecast_3m = model_fit.forecast(steps=90)
forecast_6m = model_fit.forecast(steps=180)
forecast_12m = model_fit.forecast(steps=365)

print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)
print("12개월 예측 주가:", forecast_12m)

         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  
3개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
336    0.217807
337    0.217807
338    0.217807
339    0.217807
340    0.217807
Name: predicted_mean, Length: 90, dtype: float64
6개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
426    0.217807
427    0.217807
428    0.217807
429    0.217807
430    0.217807
Name: predicted_mean, Length: 



```
# 코드로 형식 지정됨
         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  
3개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
336    0.217807
337    0.217807
338    0.217807
339    0.217807
340    0.217807
Name: predicted_mean, Length: 90, dtype: float64
6개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
426    0.217807
427    0.217807
428    0.217807
429    0.217807
430    0.217807
Name: predicted_mean, Length: 180, dtype: float64
12개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
611    0.217807
612    0.217807
613    0.217807
614    0.217807
615    0.217807
Name: predicted_mean, Length: 365, dtype: float64
```





## 데이터 분석 및 예측 결과 해석
데이터프레임
주어진 데이터프레임은 아마존 주가의 일일 데이터를 포함하고 있습니다. 각 열의 의미는 다음과 같습니다:
```
Date: 날짜
Open: 시가
High: 고가
Low: 저가
Close: 종가
Adj Close: 수정 종가
Volume: 거래량

** 예측 결과 **
ARIMA 모델을 사용하여 3개월, 6개월, 12개월 동안의 주가를 예측한 결과는 다음과 같습니다:

- 3개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.
이는 표준화된 값으로, 실제 주가로 변환하려면 역표준화를 해야 합니다.


- 6개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.
3개월 예측과 유사한 범위로, 주가가 안정적인 상승세를 보일 것으로 예상됩니다.

- 12개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.

장기적으로도 주가가 안정적인 상승세를 유지할 것으로 보입니다.
```



## Reference
<pre>
Markdown	Preview
**bold text**	bold text
*italicized text* or _italicized text_	italicized text
`Monospace`	Monospace
~~strikethrough~~	strikethrough
[A link](https://www.google.com)	A link
![An image](https://www.google.com/images/rss.png)	An image
</pre>

[참조 : 마크다운 - 작성가이드](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=Lhfnlq1Surtk)